# Libraries & Functions

In [2]:
!git config --global user.email "viktor.domazetoski@hotmail.com"
!git config --global user.name "ViktorDo1"
from huggingface_hub import notebook_login
notebook_login()

In [3]:
'''Math & Data Libraries'''
import numpy as np
import pandas as pd
import random

In [4]:
''' Miscellaneous Libraries'''
from tqdm import tqdm

In [5]:
from datasets import Dataset, DatasetDict

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
from transformers import AutoTokenizer

# Dataset Creation

### Generate requirements

In [85]:
N_desc = 500

In [86]:
 # Generate plant heights
plant_height_meters = np.round(np.random.uniform(1, 40, N_desc), 1)
plant_height_decimeters = np.round(np.random.uniform(1, 15, N_desc//2), 1)
plant_height_centimeters = np.round(np.random.uniform(1, 300, N_desc), 0)

In [87]:
 # Generate leaf sizes
leaf_length_centimeters = np.round(np.random.uniform(0, 100, N_desc), 1)
leaf_width_centimeters = np.round(np.random.uniform(0, 100, N_desc), 1)
leaf_length_milimeters = np.round(np.random.uniform(0, 30, N_desc//5), 2)
leaf_width_milimeters = np.round(np.random.uniform(0, 30, N_desc//5), 2)

In [88]:
SP_values = ["herb", "shrub", "tree", "plant", "plant", "plant", "species"]

In [89]:
# Generate plant height descriptions
plant_height_descriptions_tofill = ["The [SP] height is [NUM].",
                "The height of the [SP] is [NUM].",
                "The maximum height of the [SP] is [NUM].",
                "The [SP] is [NUM] tall.",
                "The max growth of the [SP] is [NUM]."
                ]

In [90]:
# Generate leaf length descriptions
leaf_length_descriptions_tofill = ["The [SP] leaf length is [NUM].",
                "The leaf length of the [SP] is [NUM].",
                "The maximum leaf length of the [SP] is [NUM].",
                "The leaf of [SP] is [NUM] long.",
                "The max length of the leaves of [SP] is [NUM]."
                ]

# Generate leaf width descriptions
leaf_width_descriptions_tofill = ["The [SP] leaf width is [NUM].",
                "The leaf width of the [SP] is [NUM].",
                "The maximum leaf width of the [SP] is [NUM].",
                "The leaf of [SP] is [NUM] wide.",
                "The max width of the leaves of [SP] is [NUM]."
                ]

In [91]:
# Generate descriptions containing no numbers

# Generate descriptions containing irrelevant numbers

In [92]:
plant_height_measurements = [["meters", "m"],
                             ["decimeters", "dm"],
                             ["centimeters", "cm"]]

leaf_size_measurements = [["centimeters", "cm"],
                          ["milimeters", "mm"]]

### Generate descriptions

In [93]:
# Single Descriptions
descriptions_dict = {}
questions_dict = {}
answers_dict = {}
trait_names = ["Plant Height", "Leaf Length", "Leaf Width"]
questions_list = ["What is the height of the plant?", "What is the leaf length?", "What is the leaf width?"]
trait_values_list = [[plant_height_meters, plant_height_decimeters, plant_height_centimeters],
                            [leaf_length_centimeters, leaf_length_milimeters],
                            [leaf_width_centimeters, leaf_width_milimeters]
                            ]
trait_measurements_list = [plant_height_measurements, leaf_size_measurements, leaf_size_measurements]
trait_descriptions_tofill_list = [plant_height_descriptions_tofill, leaf_length_descriptions_tofill, leaf_width_descriptions_tofill]


for trait_name, trait_values, trait_measurements, trait_descriptions_tofill, question in zip(trait_names, trait_values_list, trait_measurements_list, trait_descriptions_tofill_list, questions_list):
    descriptions_dict[trait_name] = []
    answers_dict[trait_name] = []
    questions_dict[trait_name] = []
    for i, trait_list in enumerate(trait_values):
        for j, trait_value in enumerate(trait_list):
            description = random.choice(trait_descriptions_tofill)
            sp = random.choice(SP_values)
            unit = random.choice(trait_measurements[i])
            space = " "

            answer = str(trait_value) + space + unit
            description = description.replace("[SP]", sp).replace("[NUM]", answer)

            descriptions_dict[trait_name].append(description)
            answers_dict[trait_name].append({"text": [answer], "answer_start": [description.index(answer)]})
            questions_dict[trait_name].append(question)

In [94]:
# Combined Descriptions
descriptions_dict["Combined"] = []
answers_dict["Combined"] = []
questions_dict["Combined"] = []

for i in range(N_desc):
    for trait_name in trait_names:
        for trait_name_2 in trait_names:
            if(trait_name == trait_name_2):
                continue

            index = np.random.randint(len(descriptions_dict[trait_name]))
            description_1 = descriptions_dict[trait_name][index]
            description_2 = random.choice(descriptions_dict[trait_name_2])

            description = description_1 + " " + description_2
            answer = answers_dict[trait_name][index]["text"][0]
            question = questions_dict[trait_name][index]

            descriptions_dict["Combined"].append(description)
            answers_dict["Combined"].append({"text": [answer], "answer_start": [description.index(answer)]})
            questions_dict["Combined"].append(question)

In [95]:
# Descriptions with no numbers
descriptions_dict["NoNum"] = []
answers_dict["NoNum"] = []
questions_dict["NoNum"] = []

for i in range(N_desc//2):
    for trait_descriptions_tofill in trait_descriptions_tofill_list:

        description = random.choice(trait_descriptions_tofill)
        answer = ""
        question = random.choice(questions_list)

        descriptions_dict["NoNum"].append(description)
        answers_dict["NoNum"].append({"text": [answer], "answer_start": [0]})
        questions_dict["NoNum"].append(question)

In [96]:
df_QA = pd.DataFrame()
# df_QA["type"] = np.hstack(list(descriptions_dict.keys()))
df_QA["context"] = np.hstack(list(descriptions_dict.values()))
df_QA["question"] = np.hstack(list(questions_dict.values()))
df_QA["answers"] = np.hstack(list(answers_dict.values()))
df_QA = df_QA.sample(df_QA.shape[0], random_state = 42).reset_index().drop(["index"], axis=1)
df_QA.head()

,context,question,answers
0,The tree height is 2.6 dm. The max width of th...,What is the height of the plant?,"{'text': ['2.6 dm'], 'answer_start': [19]}"
1,The height of the herb is 259.0 centimeters. T...,What is the height of the plant?,"{'text': ['259.0 centimeters'], 'answer_start'..."
2,The maximum height of the plant is 83.0 centim...,What is the height of the plant?,"{'text': ['83.0 centimeters'], 'answer_start':..."
3,The maximum leaf length of the tree is 20.2 cm...,What is the leaf length?,"{'text': ['20.2 cm'], 'answer_start': [39]}"
4,The plant leaf length is 64.4 cm.,What is the leaf length?,"{'text': ['64.4 cm'], 'answer_start': [25]}"


## Preprocess Dataset

In [97]:
indices_train, indices_test \
    = train_test_split(df_QA.index.values, test_size=0.25, random_state=42)

df_QA_train = df_QA.loc[indices_train]
df_QA_train["id"] = df_QA_train.index.astype(str)
df_QA_test = df_QA.loc[indices_test]
df_QA_test["id"] = df_QA_test.index.astype(str)

In [98]:
raw_datasets = DatasetDict()
raw_datasets["train"] = Dataset.from_pandas(df_QA_train)
raw_datasets["validation"] = Dataset.from_pandas(df_QA_test)
raw_datasets["test"] = Dataset.from_pandas(df_QA_test)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answers', 'id', '__index_level_0__'],
        num_rows: 4650
    })
    validation: Dataset({
        features: ['context', 'question', 'answers', 'id', '__index_level_0__'],
        num_rows: 1550
    })
    test: Dataset({
        features: ['context', 'question', 'answers', 'id', '__index_level_0__'],
        num_rows: 1550
    })
})

In [99]:
print("Context: ", raw_datasets["train"][0]["context"])
print("Question: ", raw_datasets["train"][0]["question"])
print("Answer: ", raw_datasets["train"][0]["answers"])

Context:  The height of the plant is 4.9 dm.
Question:  What is the height of the plant?
Answer:  {'answer_start': [27], 'text': ['4.9 dm']}


# Model Training & Evaluation

In [100]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [101]:
max_length = 384
stride = 128

def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [102]:
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset)

Map:   0%|          | 0/4650 [00:00<?, ? examples/s]

(4650, 4650)

In [103]:
def preprocess_validation_examples(examples):

    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [104]:
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/1550 [00:00<?, ? examples/s]

(1550, 1550)

In [105]:
small_eval_set = raw_datasets["validation"].select(range(100))
trained_checkpoint = "distilbert-base-cased-distilled-squad"

tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
eval_set = small_eval_set.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [106]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [107]:
import torch
from transformers import AutoModelForQuestionAnswering

eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}
trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(
    device
)

with torch.no_grad():
    outputs = trained_model(**batch)

In [108]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [109]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
    example_to_features[feature["example_id"]].append(idx)

In [110]:
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

for example in small_eval_set:
    example_id = example["id"]
    context = example["context"]
    answers = []

    for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = eval_set["offset_mapping"][feature_index]

        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Skip answers that are not fully in the context
                if offsets[start_index] is None or offsets[end_index] is None:
                    continue
                # Skip answers with a length that is either < 0 or > max_answer_length.
                if (
                    end_index < start_index
                    or end_index - start_index + 1 > max_answer_length
                ):
                    continue

                answers.append(
                    {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                )

    best_answer = max(answers, key=lambda x: x["logit_score"])
    predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})

In [111]:
import evaluate

metric = evaluate.load("squad")

In [112]:
theoretical_answers = [
    {"id": ex["id"], "answers": ex["answers"]} for ex in small_eval_set
]

In [113]:
print(predicted_answers[0])
print(theoretical_answers[0])

{'id': '5367', 'prediction_text': 'max width'}
{'id': '5367', 'answers': {'answer_start': [0], 'text': ['']}}


In [114]:
metric.compute(predictions=predicted_answers, references=theoretical_answers)

{'exact_match': 72.0, 'f1': 76.69999999999999}

In [115]:
from tqdm.auto import tqdm


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [116]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [117]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-custom_Numerical_Traits",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=False,
    push_to_hub=True,
)

In [118]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.118400,No log
2,0.000100,No log
3,0.000100,No log


TrainOutput(global_step=1746, training_loss=0.03395184197592179, metrics={'train_runtime': 1297.6076, 'train_samples_per_second': 10.751, 'train_steps_per_second': 1.346, 'total_flos': 2733817317350400.0, 'train_loss': 0.03395184197592179, 'epoch': 3.0})

In [119]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, raw_datasets["validation"])

  0%|          | 0/1550 [00:00<?, ?it/s]

{'exact_match': 100.0, 'f1': 88.06451612903226}

In [120]:
trainer.push_to_hub(commit_message="Training complete")

'https://huggingface.co/ViktorDo/bert-finetuned-custom_Numerical_Traits/tree/main/'